<a href="https://colab.research.google.com/github/sorielkobayashi/Custom-Colab-Notebooks/blob/main/automatic1111%20v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [Stable Diffusion WebUI Colab](https://github.com/ddPn08/stable-diffusion-webui-colab) by [ddPn08](https://github.com/ddpn08/)

Wiki -> https://github.com/ddPn08/automatic1111-colab/wiki

<br />

# Troubleshooting (不具合が発生したら)
1. First, check the wiki and changelog. (まずは、Wikiと変更ログを確認してください。)
  - [Wiki](https://github.com/ddPn08/automatic1111-colab/wiki)
  - [CHANGELOG | 変更ログ](#scrollTo=moDR3lrJVsE8)

2. If you still can't figure it out, open a Github issue. (それでもわからない場合はGithubのIssueを立ててください。)
  - [Github Issue](https://github.com/ddPn08/automatic1111-colab/issues/new).

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
! nvidia-smi
! nvcc -V
! free -h

Thu Dec 29 17:09:13 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   47C    P0    27W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## 1 - Setup

### 1.1 Clone repository
[Wiki / English](https://github.com/ddPn08/automatic1111-colab/wiki/1.-Setup-%7C-EN#11-clone-repository) - [Wiki / 日本語](https://github.com/ddPn08/automatic1111-colab/wiki/1.-Setup-%7C-JP#11-リポジトリのクローン)

In [3]:
%cd /content/
repository_url = "https://github.com/AUTOMATIC1111/stable-diffusion-webui"  # @param {type: "string"}
webui_branch = "master"  # @param {type: "string"}

! git clone {repository_url}
%cd /content/stable-diffusion-webui
! git checkout {webui_branch}

/content
Cloning into 'stable-diffusion-webui'...
remote: Enumerating objects: 12629, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 12629 (delta 0), reused 0 (delta 0), pack-reused 12625
Receiving objects: 100% (12629/12629), 24.71 MiB | 17.88 MiB/s, done.
Resolving deltas: 100% (8826/8826), done.
/content/stable-diffusion-webui
Already on 'master'
Your branch is up to date with 'origin/master'.


### 1.2 Setup models
[Wiki / English](https://github.com/ddPn08/automatic1111-colab/wiki/1.-Setup-%7C-EN#12-setup-models) - [Wiki / 日本語](https://github.com/ddPn08/automatic1111-colab/wiki/1.-Setup-%7C-JP#12-モデルのセットアップ)

In [4]:
# @markdown # Set up the StableDiffusion model.

# @markdown **Model Path Variables**
%cd /content/

data_dir = "/content/data"  # @param {type:"string"}

# @markdown **Optional | Download the model if it isn't already in the `{data_dir}/models` folder**

# @markdown Get huggingface access token from [here](https://huggingface.co/settings/tokens)
download_if_missing = False  # @param {type:"boolean"}
model_url = "https://huggingface.co/CompVis/stable-diffusion-v-1-4-original/resolve/main/sd-v1-4.ckpt"  # @param {type:"string"}
model_filename = ""  # @param {type:"string"}
auth_token = ""  # @param {type:"string"}

# @markdown **Optional | Use Google Drive**
mount_google_drive = True  # @param {type:"boolean"}
force_remount = False  # @param {type:"boolean"}

import os

mount_success = False
if mount_google_drive:
    from google.colab import drive

    try:
        drive_path = "/content/drive"
        drive.mount(drive_path, force_remount=force_remount)
        data_dir_gdrive = "/content/drive/MyDrive/AI"  # @param {type:"string"}
        data_dir = data_dir_gdrive
        mount_success = True
    except:
        print("...error mounting drive or with drive path variables")
        print("...reverting to default path variables")

if os.path.exists(f"{data_dir}/script.pre.sh"):
    ! chmod +x {data_dir}/script.pre.sh
    ! {data_dir}/script.pre.sh

models_path = f"{data_dir}/models"
output_path = f"{data_dir}/outputs"
config_path = f"{data_dir}/config"
extensions_file_path = f"{data_dir}/extensions.txt"

os.makedirs(models_path, exist_ok=True)
os.makedirs(output_path, exist_ok=True)
os.makedirs(config_path, exist_ok=True)
os.makedirs(f"{models_path}/Stable-diffusion", exist_ok=True)

for dir in os.listdir(models_path):
    ! rm -Rf stable-diffusion-webui/models/{dir} && ln -s {models_path}/{dir} stable-diffusion-webui/models/{dir}

! rm -f stable-diffusion-webui/outputs && ln -s {data_dir}/outputs stable-diffusion-webui/outputs
! rm -f stable-diffusion-webui/config.json && ln -s {config_path}/config.json stable-diffusion-webui/config.json
! rm -f stable-diffusion-webui/ui-config.json && ln -s {config_path}/ui-config.json stable-diffusion-webui/ui-config.json

if download_if_missing:
    if not model_filename:
        model_filename = model_url.split("/")[-1]
    if mount_google_drive and not mount_success:
        print(f"Downloading model to {models_path} due to gdrive mount error")
    elif not os.path.exists(f"{data_dir}/models/Stable-diffusion/{model_filename}"):
        os.makedirs(f"{models_path}/Stable-diffusion", exist_ok=True)
        ! curl -LJ  {model_url} -o {models_path}/Stable-diffusion/{model_filename} {'-H "Authorization: Bearer ' + auth_token + '"' if auth_token else ""}
    else:
        print("Model already downloaded, moving to next step")

/content
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
ln: target 'v2k.ckpt' is not a directory


## 2 - Advanced - Launch preferences
[Wiki / English](https://github.com/ddPn08/automatic1111-colab/wiki/2.-Launch-preferences-%7C-EN#2-launch-preferences) - [Wiki / 日本語](https://github.com/ddPn08/automatic1111-colab/wiki/2.-Launch-preferences-%7C-JP#2-起動設定)

In [5]:
# @markdown &nbsp;
# @markdown ## Command line arguments

import os

no_half = False  # @param {type:"boolean"}
no_half_vae = False # @param {type:"boolean"}
allow_code = False # @param {type:"boolean"}
no_progressbar_hiding = False  # @param {type:"boolean"}
medvram = True  # @param {type:"boolean"}
lowvram = False  # @param {type:"boolean"}
deepdanbooru = True # @param {type:"boolean"}
xformers = True  # @param {type:"boolean"}
disable_opt_split_attention = False  # @param {type:"boolean"}

# @markdown &nbsp;
custom_arguments = "" # @param {type:"string"}

run_string_with_variables = {
    "--no-half": f"{no_half}",
    "--no-half-vae": f"{no_half_vae}",
    "--allow-code": f"{allow_code}",
    "--no-progressbar-hiding": f"{no_progressbar_hiding}",
    "--medvram": f"{medvram}",
    "--lowvram": f"{lowvram}",
    "--deepdanbooru": f"{deepdanbooru}",
    "--xformers": f"{xformers}",
    "--disable-opt-split-attention": f"{disable_opt_split_attention}",
}

advanced_options = {k for (k, v) in run_string_with_variables.items() if v == "True"}

# @markdown &nbsp;
# @markdown ## Enable password authentication (Prevent other users from using the WebUI)

# @markdown &nbsp;
use_gradio_auth = True # @param {type:"boolean"}
gradio_auth_username = "soriel" # @param {type:"string"}
gradio_auth_password = "soriel" # @param {type:"string"}

# @markdown &nbsp;
# @markdown # Advanced | Network preferences

# @markdown &nbsp;
# @markdown ## Optional | Ngrok Tunnel
# @markdown Get token from [here](https://dashboard.ngrok.com/get-started/your-authtoken)
use_ngrok = False  # @param {type: "boolean"}
load_token_from_gdrive = False  # @param {type:"boolean"}
ngrok_auth_token = "2GzvUnwtKlABhSyZcrv8zblBMxE_2iWzz9QFqd2A1Hy1PQBrM"  # @param {type: "string"}
ngrok_region = "auto"  # @param ["auto", "us", "eu", "au", "ap", "sa", "jp", "in"]

if os.path.exists(f"{data_dir}/ngrok.txt") and use_ngrok:
    with open(f"{data_dir}/ngrok.txt", mode="r") as f:
        lines = f.readlines()
        if not ngrok_auth_token and len(lines) > 0:
            ngrok_auth_token = lines[0].strip()
        if ngrok_region == "auto" and len(lines) > 1:
            ngrok_region = lines[1].strip()

with open(f"{data_dir}/ngrok.txt", mode="w") as f:
    f.write(f"{ngrok_auth_token}\n{ngrok_region}")

if not ngrok_region or ngrok_region == "auto":
    ngrok_region = "us"

# @markdown &nbsp;
# @markdown ## Optional | Tailscale
# @markdown Get auth key from [here](https://login.tailscale.com/admin/settings/keys)
use_tailscale = False  # @param {type: "boolean"}
tailscale_auth_key = ""  # @param {type: "string"}

# @markdown ## Extensions
load_extensions_from_gdrive = True  # @param {type:"boolean"}
extensions = "https://github.com/yfszzx/stable-diffusion-webui-images-browser, https://github.com/DominikDoom/a1111-sd-webui-tagcomplete"  # @param {type:"string"}
extensions = list(map(str.strip, extensions.split(',')))

if load_extensions_from_gdrive and extensions_file_path:
    if os.path.exists(extensions_file_path):
        with open(extensions_file_path, mode="r") as f:
            for s in f:
                url = s.strip()
                if url not in extensions:
                    extensions.append(url)
    with open(extensions_file_path, mode="w+") as f:
        f.write("\n".join(extensions))

vars = " ".join(advanced_options)
if not use_ngrok:
    vars += " --share"
elif ngrok_auth_token and ngrok_region:
    vars += f" --ngrok {ngrok_auth_token} --ngrok-region {ngrok_region}"

if use_gradio_auth:
    vars += f" --gradio-auth {gradio_auth_username}:{gradio_auth_password}"

os.environ['COMMANDLINE_ARGS'] = f"{vars} {custom_arguments}"

## 3 - Launch WebUI
[Wiki / English](https://github.com/ddPn08/automatic1111-colab/wiki/3.-Run-%7C-EN#set-up-the-environment) - [Wiki / 日本語](https://github.com/ddPn08/automatic1111-colab/wiki/3.-Run-%7C-JP)

In [6]:
# @markdown ## Setup environment
# @markdown This may take up to 10 minutes

store_env_gdrive = False # @param{type:"boolean"}
force_reinstall_environmemt = False # @param{type:"boolean"}
gdrive_env_directory = f"{data_dir}/conda-env"
gdrive_env_file = f"{gdrive_env_directory}/env.tar.zst"

%cd /content/stable-diffusion-webui/extensions

import os
import sys

for extension in extensions:
    ! git clone {extension}
    extension_name = extension.split("/")[-1]
    ! cd {extension_name} && git fetch

%cd /content

! curl -fsSL https://pkgs.tailscale.com/stable/ubuntu/bionic.gpg | sudo apt-key add -
! curl -fsSL https://pkgs.tailscale.com/stable/ubuntu/bionic.list | sudo tee /etc/apt/sources.list.d/tailscale.list

! apt update
! apt upgrade -y
! apt install tailscale zstd -y

! rm -rf /tmp/tailscaled && mkdir -p /tmp/tailscaled && chown irc.irc /tmp/tailscaled
! rm -rf /var/run/tailscale && mkdir -p /var/run/tailscale && chown irc.irc /var/run/tailscale
! cp /var/lib/tailscaled/tailscaled.state /tmp/tailscaled/tailscaled.state && chown irc.irc /tmp/tailscaled/tailscaled.state
! nohup sudo -u irc tailscaled --tun=userspace-networking --socks5-server=localhost:1055 --state=/tmp/tailscaled/tailscaled.state --socket=/var/run/tailscale/tailscaled.sock --port 41641 &
if use_tailscale:
    ! until tailscale up --authkey {tailscale_auth_key}; do sleep 1; done

if not os.path.exists("/usr/local/bin/conda"):
    ! curl -O https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
    ! chmod +x Miniconda3-latest-Linux-x86_64.sh
    ! bash ./Miniconda3-latest-Linux-x86_64.sh -b -f -p /usr/local
    ! rm Miniconda3-latest-Linux-x86_64.sh


if os.path.exists(gdrive_env_file) and not os.path.exists("/usr/local/envs/automatic") and not force_reinstall_environmemt:
    os.makedirs("/usr/local/envs/automatic", exist_ok=True)
    ! zstd -dc {gdrive_env_file} | tar -xf - -C /usr/local/envs/automatic
    ! cd stable-diffusion-webui && conda env update -n automatic -f ./environment-wsl2.yaml
else:
    ! cd stable-diffusion-webui && conda env create -f ./environment-wsl2.yaml

install_script = """#!/bin/bash
eval "$(conda shell.bash hook)"
cd stable-diffusion-webui
conda activate automatic
conda install -y xformers -c xformers/label/dev
python3 -m pip install --upgrade tensorrt
python -c 'from launch import prepare_environment; prepare_environment()'"""
! {install_script}

os.environ["LD_LIBRARY_PATH"] = f"{os.environ['LD_LIBRARY_PATH']}:/usr/local/envs/automatic/lib"

if os.path.exists(f"{data_dir}/script.post.sh"):
    ! chmod +x {data_dir}/script.post.sh
    ! {data_dir}/script.post.sh

if store_env_gdrive and mount_google_drive and mount_success:
    import threading
    def pack():
        os.makedirs(gdrive_env_directory, exist_ok=True)
        ! tar -C /usr/local/envs/automatic -cf - . | zstd > /content/tmp.tar.zst
        ! mv /content/tmp.tar.zst {gdrive_env_file}
        print("Finish storing environment.")
    threading.Thread(target=pack).start()
    print("Started storing the conda environment in a separate thread. This will take approximately 2 minutes.")

/content/stable-diffusion-webui/extensions
Cloning into 'stable-diffusion-webui-images-browser'...
remote: Enumerating objects: 143, done.
remote: Counting objects: 100% (27/27), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 143 (delta 10), reused 17 (delta 9), pack-reused 116
Receiving objects: 100% (143/143), 41.07 KiB | 13.69 MiB/s, done.
Resolving deltas: 100% (48/48), done.
Cloning into 'a1111-sd-webui-tagcomplete'...
remote: Enumerating objects: 617, done.
remote: Counting objects: 100% (345/345), done.
remote: Compressing objects: 100% (173/173), done.
remote: Total 617 (delta 225), reused 219 (delta 167), pack-reused 272
Receiving objects: 100% (617/617), 3.72 MiB | 25.21 MiB/s, done.
Resolving deltas: 100% (294/294), done.
/content
OK
# Tailscale packages for ubuntu bionic
deb https://pkgs.tailscale.com/stable/ubuntu bionic main
Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:2 http://security.ubuntu.com/ubuntu b

In [ ]:
# @markdown # Run script
# @markdown keep in mind that this script is set to run for ever.
# @markdown > ※注意 このスクリプトは永久に実行されます。

# @markdown &nbsp;

# @markdown ### Important - click the public URL to launch WebUI in another tab
# @markdown > ### 重要 - 公開URLをクリックしてWebUIを起動してください

# @markdown ![](https://user-images.githubusercontent.com/71378929/189563599-6df78bcf-133b-41e8-a55d-8ca3783cd933.pn)

%cd /content/stable-diffusion-webui/
! git pull

run_script = """#!/bin/sh
eval "$(conda shell.bash hook)"
conda activate automatic
python launch.py"""
! {run_script}

/content/stable-diffusion-webui
Already up to date.
Python 3.10.8 (main, Nov 24 2022, 14:13:03) [GCC 11.2.0]
Commit hash: 4af3ca5393151d61363c30eef4965e694eeac15e
Installing requirements for Web UI
Launching Web UI with arguments: --medvram --xformers --deepdanbooru --share --gradio-auth soriel:soriel
A matching Triton is not available, some optimizations will not be enabled.
Error caught was: No module named 'triton'
Moving model.ckpt from /content/stable-diffusion-webui/models to /content/stable-diffusion-webui/models/Stable-diffusion.
Moving 130NET.ckpt from /content/stable-diffusion-webui/models to /content/stable-diffusion-webui/models/Stable-diffusion.
Moving sd-v1-4.ckpt from /content/stable-diffusion-webui/models to /content/stable-diffusion-webui/models/Stable-diffusion.
Moving bethsmith.ckpt from /content/stable-diffusion-webui/models to /content/stable-diffusion-webui/models/Stable-diffusion.
Moving sd-v1-5-inpainting.ckpt from /content/stable-diffusion-webui/models to /cont

# CHANGELOG (変更ログ)

## 2022/12/18 BREAKING CHANGE
モデル等のディレクトリの構造を変更しました。くわしくは[こちら](https://github.com/ddPn08/automatic1111-colab/wiki/Data-directory-%7C-JP)  
Changed the directory structure of models etc. For details [here](https://github.com/ddPn08/automatic1111-colab/wiki/Data-directory-%7C-EN)  

#EJECUTAR LO DE ABAJO POR SI VALE VERGA TODO


In [14]:
#@markdown Specify the weights directory to use (leave blank for latest)
WEIGHTS_DIR = "" #@param {type:"string"}
if WEIGHTS_DIR == "":
    from natsort import natsorted
    from glob import glob
    import os
    WEIGHTS_DIR = natsorted(glob(OUTPUT_DIR + os.sep + "*"))[-1]
print(f"[*] WEIGHTS_DIR={WEIGHTS_DIR}")

[*] WEIGHTS_DIR=/content/drive/MyDrive/stable_diffusion_weights/c0wc0wc0w/1500


In [16]:
!wget -q https://github.com/ShivamShrirao/diffusers/raw/main/scripts/convert_diffusers_to_original_stable_diffusion.py
ckpt_path ="/content/drive/MyDrive/stable_diffusion_weights/c0wc0wc0w/1500"

half_arg = ""
#@markdown  Whether to convert to fp16, takes half the space (2GB).
fp16 = False #@param {type: "boolean"}
if fp16:
    half_arg = "--half"
!python convert_diffusers_to_original_stable_diffusion.py --model_path $WEIGHTS_DIR  --checkpoint_path $ckpt_path $half_arg
print(f"[*] Converted ckpt saved at {ckpt_path}")

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/site-packages/torch/__init__.py", line 172, in _load_global_deps
    ctypes.CDLL(lib_path, mode=ctypes.RTLD_GLOBAL)
  File "/usr/local/lib/python3.10/ctypes/__init__.py", line 374, in __init__
    self._handle = _dlopen(self._name, mode)
OSError: /usr/local/lib/python3.10/site-packages/torch/lib/../../nvidia/cublas/lib/libcublas.so.11: symbol cublasLtGetStatusString version libcublasLt.so.11 not defined in file libcublasLt.so.11 with link time reference

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/content/stable-diffusion-webui/convert_diffusers_to_original_stable_diffusion.py", line 9, in <module>
    import torch
  File "/usr/local/lib/python3.10/site-packages/torch/__init__.py", line 217, in <module>
    _load_global_deps()
  File "/usr/local/lib/python3.10/site-packages/torch/__init__.py", line 178, in _load_global_deps
    _preload_cuda_deps()
 